# Qualidade dos dados e Pré-processamento 

![alt text](https://joaomrcarvalho.github.io/images/images.png)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
%matplotlib inline

## Valores em falta (missing values)

### Detectar valores em falta

In [ ]:
data = pd.read_csv('https://joaomrcarvalho.github.io/datasets/dm/missed_phone_data.csv',index_col='index')

data.head()

In [ ]:
data.isnull().head()

In [ ]:
print(data.isnull().any(axis=0))

print('------------------------------------------------------------------------------------------')

print(data.isnull().any(axis=1))

In [ ]:
print(data.isnull().any(axis=0).value_counts())


print('----------------------------------------------------------------------------------------------')

print(data.isnull().any(axis=1).value_counts())

In [ ]:
nan_data  = data[data.isnull().any(axis=1)]
print(nan_data.head())

In [ ]:
complete_data  = data[data.notnull().all(axis=1)]

print(complete_data.head())

### Lidar com valores em falta

In [ ]:
incomplet_data = pd.read_csv('https://joaomrcarvalho.github.io/datasets/dm/missed_phone_data.csv', index_col='index')
complet_data = pd.read_csv('https://joaomrcarvalho.github.io/datasets/dm/phone_data.csv', index_col='index')

miss_indexes =  data[incomplet_data.isnull().any(axis=1)].index

print(miss_indexes)

In [ ]:
print(incomplet_data.columns[incomplet_data.isnull().any(axis=0)])

In [ ]:
print(incomplet_data['duration'].head(5))

incomplet_data['duration'].fillna(incomplet_data['duration'].mean(),inplace=True)

In [ ]:
duration_true = complet_data.iloc[miss_indexes]['duration']
duration_completed = incomplet_data.iloc[miss_indexes]['duration']

print(abs(duration_true - duration_completed).sum()/len(duration_completed))

In [ ]:
print(incomplet_data['item'].head(5))

print("-------------------------")

print(incomplet_data['item'].mode())

print("-------------------------")

incomplet_data['item'].fillna(incomplet_data['item'].mode,inplace=True)

In [ ]:
item_true = complet_data.iloc[miss_indexes]['item']
item_completed = incomplet_data.iloc[miss_indexes]['item']

print(len(item_true.loc[item_true == item_completed])/len(item_completed))

In [ ]:
incomplet_data = pd.read_csv('https://joaomrcarvalho.github.io/datasets/dm/missed_phone_data.csv',index_col='index')

out_data = incomplet_data.loc[incomplet_data['duration'].notnull()]

In [ ]:
group = out_data.groupby('item').mean()['duration']

print(group)

print('------------------------------')

print(group.loc['call'])

In [ ]:
# tendo em conta os resultados anteriores,
# podemos  preencher os dados em falta:
for i, value in enumerate(incomplet_data['item']):
    if pd.isnull(incomplet_data['duration'][i]) == True and  pd.isnull(incomplet_data['item'][i]) == False:
        incomplet_data.loc[i,['duration']] = group.loc[value]
        
incomplet_data['duration'].fillna(incomplet_data['duration'].mean(),inplace=True)

**Sugestão de exercício**: Completar o código acima substituindo o preenchimento de valores em falta pela variável agrupada por outra coluna, de forma a obter um resultado mais fidedigno. 

In [ ]:
# vamos carregar os registos da coluna 'duration' com missing values
duration_true = complet_data.iloc[miss_indexes]['duration']
duration_completed = incomplet_data.iloc[miss_indexes]['duration']

print(abs(duration_true - duration_completed).sum()/len(duration_completed))

In [ ]:
incomplet_data = pd.read_csv('https://joaomrcarvalho.github.io/datasets/dm/missed_phone_data.csv', index_col='index')


out_data = incomplet_data.loc[incomplet_data[['network_type','network']].notnull().any(axis=1)]

In [ ]:
print(out_data.head())

print("--------------------------------")

group_main = out_data.groupby(['network_type','network'],as_index=True).agg(lambda x: x.value_counts().index[0])['item']

# vamos agora fazer o mesmo procedimento, mas fazendo os grupos coluna a coluna individualmente
# estes valores vão ser úteis no caso em que um dos valores seja NaN
group_network_type =  out_data.groupby(['network_type'],as_index=True).agg(lambda x:x.value_counts().index[0])['item']
group_network =  out_data.groupby(['network'],as_index=True).agg(lambda x:x.value_counts().index[0])['item']


print(group_main)

print("--------------------------------")

print(group_network_type)

print("--------------------------------")

print(group_network)

In [ ]:
# com esses três grupos, podemos agora substituir os valores em falta:
for i,nt in enumerate(incomplet_data[['network_type','network']].values):
    if pd.isnull(incomplet_data['item'][i]) == True: 
        
        if pd.notnull(incomplet_data['network_type'][i]) == True and pd.notnull(incomplet_data['network'][i]) == True:
            incomplet_data.loc[i,['item']] = group_main.loc[incomplet_data['network_type'][i]][incomplet_data['network'][i]]
       
        elif pd.notnull(incomplet_data['network_type'][i]) == False and pd.notnull(incomplet_data['network'][i]) == True:
             incomplet_data.loc[i,['item']] = group_network.loc[incomplet_data['network'][i]]
        
        elif pd.notnull(incomplet_data['network_type'][i]) == True and pd.notnull(incomplet_data['network'][i]) == False:
             incomplet_data.loc[i,['item']] = group_network.loc[incomplet_data['network_type'][i]]
    

incomplet_data['item'].fillna(incomplet_data['item'].mode(), inplace=True)

In [ ]:
item_true = complet_data.iloc[miss_indexes]['item']
item_completed = incomplet_data.iloc[miss_indexes]['item']


print(len(item_true.loc[item_true == item_completed])/len(item_completed))

## Dados redundantes

In [ ]:
# !wget 'https://joaomrcarvalho.github.io/datasets/dm/Calls_for_Service_2015.zip'
# !unzip 'Calls_for_Service_2015.zip'

data = pd.read_csv('Calls_for_Service_2015.csv')

data.head()

In [ ]:
print(data.duplicated().head(10))

In [ ]:
print(data.duplicated().value_counts())

In [ ]:
print(data.drop_duplicates().head())

In [ ]:
data['NOPD_Item'].duplicated().head(10)

In [ ]:
print(data[['Priority', 'InitialType']].duplicated().value_counts())

In [ ]:
data = data[data[['Priority', 'InitialType']].duplicated() == False]

print(len(data))

## Lidar com ruído nos dados

In [ ]:
data = pd.read_csv('https://joaomrcarvalho.github.io/datasets/dm/house_data.csv',index_col=0)
data.head(5)

**Sugestão de Exercício:** Preencher os valores do dataset 'house_data.csv' e justificar o motivo para os métodos utilizados.

### Encontrar ruído através da visualização de dados

In [ ]:
data['SalePrice'].plot.box()
print(data['SalePrice'].describe())

In [ ]:
data.boxplot(column='SalePrice', by='SaleCondition')

In [ ]:
data.boxplot(column='SalePrice', by='GarageCars')

In [ ]:
data.boxplot(column='SalePrice', by=['GarageCars','SaleCondition'],figsize=(25,7))

In [ ]:
data.plot.scatter(x='LotArea',y='SalePrice')

In [ ]:
data.plot.scatter(x='LotArea',y='SalePrice', c= data['GarageCars'])

In [ ]:
select_data = data[(data['SalePrice'] < 600000) & (data['LotArea'] < 50000)]

select_data.plot.scatter(x='LotArea',y='SalePrice',c=select_data['GarageCars'])

**Sugestão de Exercício:** Utilizar o box plot e scatter plot aplicados também a outras variáveis, para eliminar ruídos e apresentar a média dos preços das transações de imóveis.

### Detectar ruído através de índices e fórmulas estatísticas

## Z-score

In [ ]:
data = pd.read_csv('https://joaomrcarvalho.github.io/datasets/dm/house_data.csv',index_col=0)

In [ ]:
from scipy.stats import zscore

data['SalePrice_standartized'] = zscore(data['SalePrice'],axis=0)

print(data['SalePrice_standartized'].head())

In [ ]:
noise_data_super = data[data['SalePrice_standartized'] >= 3]['SalePrice']

print(noise_data_super)

In [ ]:
print(noise_data_super.min())

In [ ]:
noise_data_inf = data[data['SalePrice_standartized'] <= -3]['SalePrice']

print(noise_data_inf)

In [ ]:
data['std_SalePrice_by_group'] = data.groupby('GarageCars')['SalePrice'].transform(zscore)

In [ ]:
noise_data = data[abs(data['std_SalePrice_by_group']) >= 3]['SalePrice']

print(noise_data)

## IQR

In [ ]:
primeiro_quartil = data['SalePrice'].quantile(0.25)

terceiro_quartil = data['SalePrice'].quantile(0.75)

print('primeiro quartil: {}'.format(primeiro_quartil))
print('terceiro quartil: {}'.format(terceiro_quartil))

In [ ]:
clean_data = data[~((data['SalePrice'] < primeiro_quartil) | (data['SalePrice'] > terceiro_quartil))]
# Nota: o símbolo "~" indica negação

print(clean_data.head())

In [ ]:
clean_data.plot.scatter(x='LotArea',y='SalePrice')

In [ ]:
percentil_5 = data['SalePrice'].quantile(0.05)

percentil_95 = data['SalePrice'].quantile(0.95)

print('percentil 5: {}'.format(percentil_5))
print('percentil 95: {}'.format(percentil_95))

clean_data = data[~((data['SalePrice'] < percentil_5) | (data['SalePrice'] > percentil_95))]

clean_data.plot.scatter(x='LotArea',y='SalePrice')